<a href="https://colab.research.google.com/github/multi-proj/API/blob/master/Soccer_Team/%5Bsey%5DTeam_Sheet_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from selenium import webdriver

In [74]:
!pip install selenium
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (73.0.3683.86-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [0]:
!pip install -U -q Pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
url = 'https://drive.google.com/open?id=1dwaiV1_fIVBGHGeVKnRGXMgw-POve3I3'
id = url.split('=')[1]

In [77]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd.get("https://www.premierleague.com/players?se=79&cl=21")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


## **지난 경기 기록 긁어오기**

In [78]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
match_wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
match_wd.get("https://www.premierleague.com/clubs/21/Tottenham-Hotspur/results?co=1&se=79")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


In [0]:
# match_tag = match_wd.find_elements_by_tag_name('li')

match_num_01 = []

for i in range(1, 39):
    a = i * 3
    number = '#mainContent > div:nth-child(6) > div > div.wrapper.col-12.active > div.col-12 > section > div:nth-child(%d) > ul > li > div' % a
    b = match_wd.find_element_by_css_selector(number)
    
    match_num_01.append(b)

In [0]:
match_num_02 = []

for i in range(0, len(match_num_01)):
    a = match_num_01[i].get_attribute('data-matchid')
    match_num_02.append(a)
    
match_num_02 = list(map(int,match_num_02))
match_num_02

In [0]:
match_num_03 = []
for i in match_num_02:
    a = 'https://www.premierleague.com/match/%d' % i
    match_num_03.append(a)
    
match_num_03

---

## **경기별 선수 명단 긁어오기**

In [0]:
# https://www.premierleague.com/match/22720
from bs4 import BeautifulSoup
import urllib.request
import urllib.parse

# web_url에 원하는 웹의 URL을 넣어주시면 됩니다.

players = []

for i in match_num_03:

    with urllib.request.urlopen(i) as response:
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')
        players.append(soup.find_all('div', 'matchLineupTeamContainer'))

In [0]:
players # 이중리스트구조 상태

In [0]:
# 리스트 구조 변경 (이중리스트 -> 단일리스트)
players = sum(players, [])

players # 구조변경 확인하기

In [0]:
players_21 = []


# Tottenham(21)만 추출
for i in range(0, len(players)):
    if players[i].get_attribute_list('data-team-id')[0] == '21':
        players_21.append(players[i])
                
players_21

In [233]:
import pandas as pd
import numpy as np

data = []

df = pd.DataFrame(data, columns = ['Name', 'Goal'])
df

# 데이터프레임에 데이터 추가하는 법
# df.loc[j, 'Name'] = players_name
# df.loc[j, 'Goal'] = players_goal

,Name,Goal


In [0]:
# Tottenham 경기정보만 불러오기

players_name = []

for i in players_21:
    players_name.append(i.find_all('div', 'name'))
    
players_name

In [235]:
# 이름 추출
players_goal = []

players_name_ex = []
for i in range(0, len(players_name)):
    temp1 = []
    
    

    for j in players_name[i]:
        # j = <div class="name"> Robert Elliot <div class="playerLineupEventContainer-2,214"></div></div>
        
        temp2 = []

        # 이름 정규화
        j = j.get_text()
        temp2.append((j.split(' '))[1:3])
        j = sum(temp2, [])
        j = " ".join(j)

        temp1.append(j)
        
    players_name_ex.append(temp1)
    
    # structuring
    df.loc[i, 'Name'] = temp1
    
##### designed structure list
df.loc[0]['Name']

['Hugo Lloris',
 'Eric Dier',
 'Kyle Walker-Peters',
 'Danny Rose',
 'Toby Alderweireld',
 'Victor Wanyama',
 'Erik Lamela',
 'Lucas Moura',
 'Moussa Sissoko',
 'Christian Eriksen',
 'Harry Kane',
 'Michel Vorm',
 'Ben Davies',
 'Juan Foyth',
 'Davinson Sánchez',
 'Dele Alli',
 'Son Heung-Min',
 'Fernando Llorente']

In [0]:
players_goal = []


for j in range(0, len(players_21)):
    # Goal수 추출
    players_goal = int(len(players_21[j].find_all("span", class_="icn ball-small")))

    df.loc[j, 'Goal'] = players_goal

In [237]:
df.loc[1]

Name    [Hugo Lloris, Davinson Sánchez, Kieran Trippie...
Goal                                                    1
Name: 1, dtype: object

#### 정규식
    * 출처1: http://melonicedlatte.com/python/2017/02/23/152309.html
    * 출처2: https://niceman.tistory.com/156
    
    

```
players_name = []

for i in range(0, len(players)) :
    if (players[i].get_attribute_list('data-team-id')[0] == '21') :
        for j in range(0, len(a)):
            print(len(a[j].find_all("span", class_="icn ball-small")))



lstrip_players_name = []

for i in range(0, len(players_name)):
    a = player_name[i].lstrip()
    lstrip_players_name.append(a)
    
lstrip_player_name




re_players_name = []
fin_players_name = []

import re

for i in range(0, len(lstrip_players_name)):
    text = lstrip_players_name[i]
    parse = re.sub('[' '][0-9][0-9][-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', text)
    re_players_name.append(parse)

for j in range(0, len(re_players_name)):
    a = re_players_name[j].rstrip()
    fin_players_name.append(a)
        
fin_players_name

#     for j in range(0, len(re_player_name)):
#         a = re_player_name[j].rstrip()
#         fin_player_name.append(a)```

